In [1]:
import sys
import os
import re

if not os.path.exists("books"):
    os.makedirs("books")

In [27]:
# Add desired pdf to book directory, chance title variable to appriate pdf title
title = "Flow-Mihaly_Csikszentmihalyi"


In [ ]:
# Then copy paste in the book's table of contents in this generated text file
with open("books/"+title+"_toc.txt", "w+") as file:
    file.write("*Copy Paste in the Book's Table of Contents here")

In [12]:
# This does basic formatting of the pasted in table of contents. 

def trim_contents(title):
    with open("books/"+title+"_toc.txt", "r") as file:
        text = file.read()
        text = re.sub(r"(\.+)", "", text)
        # text = re.sub(r"(/\n:)", ":", text) #TODO edit colons to stay on same line
        text = re.sub(r"( \d+)", r"\1 \n", text)

    lines = [line.rstrip() for line in text.split("\n") if line.strip()]

    with open("books/"+title+"_toc_edited.txt", "w+") as file:
        for line in lines:
            file.write(line.replace("\t", " ").rstrip()+"\n")
            
trim_contents(title)

In [ ]:
# Preview the changes in the _edited.txt file, and format so that it matches the _edited_sample.txt file
# Essentially, calculate first_page = y - z, where 
# y is the page number in the pdf file structure of the first page int he first chapter
# and z is the actual page number of the first page (usually 1)

# Also, each space means that heading is a subheading of a previous heading. Please look at the example files and books for more info.


first_page = 4

In [43]:
# This method generates the chapter titles and their pages according to the pdf file strucutre

def update_pages(title, first_page):
    tit_list = []
    with open("books/"+title+"_toc_edited.txt", "r") as file:
        for line in file.readlines():
            try:
                temp = line.rstrip().rsplit(" ", 1)
                tit_list.append([temp[0], int(temp[1].strip())])
            except Exception as e:
                print(e)
                print(line)
                print(temp)
        lag = first_page - tit_list[0][1]
        
    for i in range(len(tit_list)):
        tit_list[i][1] += lag

    return tit_list

    
titles = update_pages(title, first_page)

In [44]:
# generates meta content to be used for the final pdf from the previous generated titles and their pages

def gen_content(titles):
    contents = []
    for i in titles:
        level = len(i[0]) - len(i[0].lstrip())
        contents.append("BookmarkBegin")
        contents.append("BookmarkTitle: "+i[0].lstrip())
        contents.append("BookmarkLevel: "+ str(level+1))
        contents.append("BookmarkPageNumber: "+ str(i[1]))
    return contents

    
new_meta_content = gen_content(titles)

In [45]:
#generates current meta file for the pdf using pdftk, removes existing bookmarks, adds in proper bookmarks from previous file

def update_meta_content(title, contents):
    book = "books/"+title+".pdf"
    book1 = "books/"+title+"1.pdf"
    meta = "books/"+title+"_meta.txt"
    os.system("pdftk " + book + " dump_data output books/" + title + "_meta.txt")
    
    with open(meta, "r") as file:
        meta_lines = []
        for ind, l in enumerate(file.readlines()):
            if "Bookmark" not in l:
                meta_lines.append(l.strip())
            if "NumberOfPages" in l:
                page_index = ind
        final_meta = meta_lines[:page_index]+ contents + meta_lines[page_index:]

    with open(meta, "w+") as file:
        for l in final_meta:
            file.write(l+"\n")

    os.system("pdftk " + book + " update_info " + meta + " output " + book1)
    os.system("rm " + meta)
    os.system("mv " + book1 + " " + book)

update_meta_content(title, new_meta_content)